<a href="https://colab.research.google.com/github/ADeogratias/spam_or_not_spam_model/blob/main/Spam_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ls

Dataset/  sample_data/  vec_tokenizer.pickel


In [ ]:
df = pd.read_csv("Dataset/spam_or_not_spam.csv").astype(str)

In [ ]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [ ]:
import re
import nltk
from keras.layers import Flatten, Dense
from keras.layers import Embedding,LSTM
from keras.models import Sequential
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df['label'].value_counts()

0    2500
1     500
Name: label, dtype: int64

we have 500 hams and 2500 spams

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

In [ ]:
X = df['email'].fillna(' ')
y = df['label']

In [ ]:
# X = df['email'].values
# y = df['label'].values

In [ ]:
def preprocess(text):
    text = text.str.lower()
    text = text.str.replace(r'\r', ' ')
    text = text.str.replace(r'\n', ' ')
    text = text.str.replace('[^a-zA-Z0-9]', ' ')
    return text

In [ ]:
x = preprocess(X)
x = x.values
y = y.values

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state=0, shuffle = True, stratify= df['label'])
# xtrain

In [ ]:
# xtest

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenizer_sequences(X):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = len(tokenizer.word_index) + 1
    
    return tokenizer, sequences

In [ ]:
tokenizer, sequences = tokenizer_sequences(xtrain)

In [ ]:
# integer encode the documents
train_set_encoded = tokenizer.texts_to_sequences(xtrain)
test_set_encoded = tokenizer.texts_to_sequences(xtest)
print(train_set_encoded[0:2])

[[4617, 39, 21, 2032, 633, 3, 45, 11, 8, 232, 6, 667, 181, 94, 36, 4617, 718, 200, 304, 23, 410, 6, 851, 1170, 379, 13, 257, 20, 78, 4617, 1, 4617, 634, 39, 1238, 870, 46, 81, 1, 3, 76, 12, 2, 181, 2, 4617, 1171, 317, 3, 19, 3, 285, 2, 95, 45, 12, 179, 121, 302, 4617, 3, 20, 6, 466, 4303, 3, 103, 614, 12, 49, 6, 536, 41, 39, 4617, 109, 285, 13, 14, 6, 200, 18, 1073, 5, 799, 1, 1381, 4617, 2033, 1330, 58, 11943, 12, 819, 1331, 81, 70, 53, 8, 97, 7, 40, 568, 600, 34, 11, 371, 5, 436, 3, 8284, 51, 1412, 11944, 8, 102, 506, 39, 437, 2, 1469, 6509, 6510, 213, 43, 5, 133, 8285, 16016, 11, 39, 8286, 1470, 14, 5893, 339, 60, 101, 4, 492, 16, 168, 2604, 70, 13, 102, 2076, 51, 183, 10, 39, 98, 1870, 39, 71, 23, 121, 3, 45, 15, 4, 506, 81, 2, 5894, 800, 58, 18, 36, 16017, 495, 5, 5895, 2, 1526, 16018, 39, 4618, 1870, 1720, 60, 15, 22, 204, 388, 9, 275, 204, 388, 9, 16, 801, 5, 788, 9, 41, 8, 48, 6, 181, 22, 8287, 2209, 190, 8288, 25, 39, 98, 425, 69, 1871, 51, 39, 583, 3, 2, 1332, 5, 3472, 51, 38

In [ ]:
import pickle
# saving
with open('vec_tokenizer.pickel', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
max_length = 6
padded_train_set = pad_sequences(train_set_encoded, maxlen=max_length, padding='post')
padded_test_set = pad_sequences(test_set_encoded, maxlen=max_length, padding='post')
# print(padded_test_set)

In [ ]:
# load the whole embedding into memory
embeddingz = {}
f = open('/content/glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddingz[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddingz))

Loaded 400000 word vectors.


In [ ]:
#weight matrix of words in training docs
embedding_matrix = np.zeros((sequences, 100))
len(embedding_matrix)
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddingz.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector 

In [ ]:
opt = Adam(lr = 0.001)
loss = 'binary_crossentropy'

In [ ]:
model = Sequential()
model.add(Embedding(sequences, 100, weights=[embedding_matrix], input_length=6))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 6, 100)            3032200   
_________________________________________________________________
flatten_3 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                6010      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 3,038,221
Trainable params: 3,038,221
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=200, verbose=1)

Epoch 1/200
75/75 [==============================] - 2s 31ms/step - loss: 0.3516 - accuracy: 0.8508
Epoch 2/200
75/75 [==============================] - 2s 30ms/step - loss: 0.2327 - accuracy: 0.9067
Epoch 3/200
75/75 [==============================] - 2s 30ms/step - loss: 0.1659 - accuracy: 0.9413
Epoch 4/200
75/75 [==============================] - 2s 31ms/step - loss: 0.1196 - accuracy: 0.9613
Epoch 5/200
75/75 [==============================] - 2s 30ms/step - loss: 0.0926 - accuracy: 0.9679
Epoch 6/200
75/75 [==============================] - 2s 30ms/step - loss: 0.0735 - accuracy: 0.9700
Epoch 7/200
75/75 [==============================] - 2s 30ms/step - loss: 0.0618 - accuracy: 0.9783
Epoch 8/200
75/75 [==============================] - 2s 31ms/step - loss: 0.0559 - accuracy: 0.9771
Epoch 9/200
75/75 [==============================] - 2s 30ms/step - loss: 0.0492 - accuracy: 0.9800
Epoch 10/200
75/75 [==============================] - 2s 30ms/step - loss: 0.0486 - accuracy: 0.9775

In [ ]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.3324663043022156 test accuracy:  92.00000166893005 %


In [ ]:
opt = Adam(lr = 0.0001)
loss = 'binary_crossentropy'

# Lets optimize our model
model = Sequential()
model.add(Embedding(sequences, 100, weights=[embedding_matrix], input_length=6))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 6, 100)            3032200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 3,113,621
Trainable params: 3,113,621
Non-trainable params: 0
___________________________________________

In [ ]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=60, verbose=1)

Epoch 1/60
75/75 [==============================] - 3s 35ms/step - loss: 0.4824 - accuracy: 0.8213
Epoch 2/60
75/75 [==============================] - 3s 36ms/step - loss: 0.4121 - accuracy: 0.8325
Epoch 3/60
75/75 [==============================] - 3s 36ms/step - loss: 0.3824 - accuracy: 0.8346
Epoch 4/60
75/75 [==============================] - 3s 34ms/step - loss: 0.3477 - accuracy: 0.8458
Epoch 5/60
75/75 [==============================] - 3s 37ms/step - loss: 0.3042 - accuracy: 0.8771
Epoch 6/60
75/75 [==============================] - 3s 34ms/step - loss: 0.2695 - accuracy: 0.9000
Epoch 7/60
75/75 [==============================] - 3s 34ms/step - loss: 0.2398 - accuracy: 0.9167
Epoch 8/60
75/75 [==============================] - 3s 35ms/step - loss: 0.2230 - accuracy: 0.9187
Epoch 9/60
75/75 [==============================] - 3s 34ms/step - loss: 0.2027 - accuracy: 0.9258
Epoch 10/60
75/75 [==============================] - 3s 34ms/step - loss: 0.1888 - accuracy: 0.9283
Epoch 11/

In [ ]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.2654097378253937 test accuracy:  93.83333325386047 %


In [ ]:
# save model
model.save('spam_model.sav')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: spam_model.sav/assets


In [ ]:
# load model
from keras.models import load_model
model = load_model( 'spam_model.sav' )

In [ ]:
# downloadin the model and use it in Heroku
from google.colab import files
files.download('spam_model.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>